In [1]:
"""
Author - Yogesh Agrawal
Email -  Yogesh.agrawal@mindtree.com
"""

Out[ 5 ]: '\nAuthor - Yogesh Agrawal\nEmail - Yogesh.agrawal@mindtree.com\n'

In [2]:
"""
Loading important package of spark 
"""
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer,VectorAssembler,QuantileDiscretizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
"""
Spark session creater 
"""

spark = SparkSession \
        .builder \
        .appName('Titanic') \
        .getOrCreate()

In [4]:
"""
Load data function for loading data..
@param - 
        path - path of file
        header_value - header value, incase true first row will be header
        
@return - dataframe of loaded intended data.
"""

def load_data(path,header_value):
  df = spark.read.csv(path,inferSchema=True,header=header_value)
  return df

In [5]:
df = load_data('/FileStore/tables/titanic_train.csv',True)
df_test = load_data('/FileStore/tables/titanic_test.csv',True)

In [6]:
df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
PassengerId|Survived|Pclass| Name| Sex| Age|SibSp|Parch| Ticket| Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
 1| 0| 3|Braund, Mr. Owen ...| male|22.0| 1| 0| A/5 21171| 7.25| null| S|
 2| 1| 1|Cumings, Mrs. Joh...|female|38.0| 1| 0| PC 17599|71.2833| C85| C|
 3| 1| 3|Heikkinen, Miss. ...|female|26.0| 0| 0|STON/O2. 3101282| 7.925| null| S|
 4| 1| 1|Futrelle, Mrs. Ja...|female|35.0| 1| 0| 113803| 53.1| C123| S|
 5| 0| 3|Allen, Mr. Willia...| male|35.0| 0| 0| 373450| 8.05| null| S|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 5 rows

In [7]:
"""
check null value each column wise.
@param - 
          df - a dataframe 
"""
def check_column_null(df):
  df.select([count(when(col(c).isNull(),c)).alias(c) for c in df.columns]).show()

In [8]:
check_column_null(df)
check_column_null(df_test)

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-------+-----------+-----+
PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|Initial|Family_Size|Alone|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-------+-----------+-----+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 687| 0| 0| 0| 0|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+-------+-----------+-----+

+-----------+------+----+---+---+-----+-----+------+----+-----+--------+-------+-----------+-----+
PassengerId|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|Initial|Family_Size|Alone|
+-----------+------+----+---+---+-----+-----+------+----+-----+--------+-------+-----------+-----+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 327| 0| 0| 0| 0|
+-----------+------+----+---+---+-----+-----+------+----+-----+--------+-------+-----------+-----+

In [9]:
"""
Generate feature column in dataframe based on specific logic

@param - 
         df - dataframe for operation.
"""
def feature_generation(df):
  df = df.withColumn("Initial",regexp_extract(col("Name"),"([A-Za-z]+)\.",1))
  df = df.replace(['Mlle','Mme', 'Ms', 'Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
               ['Miss','Miss','Miss','Mr','Mr',  'Mrs',  'Mrs',  'Other',  'Other','Other','Mr','Mr','Mr'])
  
  df = df.withColumn("Family_Size",col('SibSp')+col('Parch'))
  df = df.withColumn('Alone',lit(0))
  df = df.withColumn("Alone",when(df["Family_Size"] ==0, 1).otherwise(df["Alone"]))
  
  return df

In [10]:
df = feature_generation(df)
df_test = feature_generation(df_test)

In [11]:
"""
Impute Age based on Age mean of specific gender. ex for male mean is 46 update all null male row with 46, similarly for others

@param - 
        df - dataframe for operation

@return -
       df - with imputed value

"""

def Age_impute(df):
  Age_mean = df.groupBy("Initial").avg('Age')
  Age_mean = Age_mean.withColumnRenamed('avg(Age)','mean_age')
  Initials_list = Age_mean.select("Initial").rdd.flatMap(lambda x: x).collect()
  Mean_list = Age_mean.select("mean_age").rdd.flatMap(lambda x: x).collect()
  
  for i,j in zip(Initials_list,Mean_list):
      df = df.withColumn("Age",when((df["Initial"] == i) & (df["Age"].isNull()), j).otherwise(df["Age"]))
      
  return df

In [12]:
"""
Impute Embark based on mode of embark column
@param - 
        df - dataframe for operation

@return -
       df - with imputed value

"""

def Embark_impute(df):
  mode_value = df.groupBy('Embarked').count().sort(col('count').desc()).collect()[0][0]
  df = df.fillna({'Embarked':mode_value})
  return df

In [13]:
"""
Impute Fare based on the class which he/she had sat ex: class 3rd has mean fare 9 and null fare belong to 3rd class so fill 9
@param - 
        df - dataframe for operation

@return -
       df - with imputed value

"""

def Fare_impute(df):
  Select_pclass = df.filter(col('Fare').isNull()).select('Pclass')
  if Select_pclass.count() > 0:
    Pclass = Select_pclass.rdd.flatMap(lambda x: x).collect()
    for i in Pclass:
      mean_pclass_fare = df.groupBy('Pclass').mean().select('Pclass','avg(Fare)').filter(col('Pclass')== i).collect()[0][1]
      df = df.withColumn("Fare",when((col('Fare').isNull()) & (col('Pclass') == i),mean_pclass_fare).otherwise(col('Fare')))
  return df
    

In [14]:
'''
combining all column imputation together..

@param - 
      df - a dataframe for operation.

@return - 
      df - dataframe with imputed value.
 
'''

def all_impute_together(df):
  df = Age_impute(df)
  df = Embark_impute(df)
  df = Fare_impute(df)
  return df

In [15]:
df = all_impute_together(df)
df_test = all_impute_together(df_test)

In [16]:
"""
Drop column from dataframe
@param -
       df - dataframe 
       col_name - name of column which need to be dropped.
@return -
       df - a dataframe except dropped column
"""

def drop_column(df,col_list):
  for i in col_list:
      df = df.drop(col(i))
  return df

In [17]:
df = drop_column(df,['Cabin','Name','Ticket','Family_Size','SibSp','Parch'])
df_test = drop_column(df,['Cabin','Name','Ticket','Family_Size','SibSp','Parch'])

In [18]:
df_test.show()

+-----------+--------+------+------+------------------+-------+--------+-------+-----+
PassengerId|Survived|Pclass| Sex| Age| Fare|Embarked|Initial|Alone|
+-----------+--------+------+------+------------------+-------+--------+-------+-----+
 1| 0| 3| male| 22.0| 7.25| S| Mr| 0|
 2| 1| 1|female| 38.0|71.2833| C| Mrs| 0|
 3| 1| 3|female| 26.0| 7.925| S| Miss| 1|
 4| 1| 1|female| 35.0| 53.1| S| Mrs| 0|
 5| 0| 3| male| 35.0| 8.05| S| Mr| 1|
 6| 0| 3| male| 32.73960880195599| 8.4583| Q| Mr| 1|
 7| 0| 1| male| 54.0|51.8625| S| Mr| 1|
 8| 0| 3| male| 2.0| 21.075| S| Master| 0|
 9| 1| 3|female| 27.0|11.1333| S| Mrs| 0|
 10| 1| 2|female| 14.0|30.0708| C| Mrs| 0|
 11| 1| 3|female| 4.0| 16.7| S| Miss| 0|
 12| 1| 1|female| 58.0| 26.55| S| Miss| 1|
 13| 0| 3| male| 20.0| 8.05| S| Mr| 1|
 14| 0| 3| male| 39.0| 31.275| S| Mr| 0|
 15| 0| 3|female| 14.0| 7.8542| S| Miss| 1|
 16| 1| 2|female| 55.0| 16.0| S| Mrs| 1|
 17| 0| 3| male| 2.0| 29.125| Q| Master| 0|
 18| 1| 2| male| 32.73960880195599| 13.0| S| Mr| 1|
 19| 0| 3|female| 31.0| 18.0| S| Mrs| 0|
 20| 1| 3|female|35.981818181818184| 7.225| C| Mrs| 1|
+-----------+--------+------+------+------------------+-------+--------+-------+-----+
only showing top 20 rows

In [19]:
# df.count()
# df.select("Survived","Pclass","Embarked").show()
#df.groupBy('Survived').count().show()
#df.groupBy(["Sex","Survived"]).count().show()
#df.groupBy("Pclass","Survived").count().show()
#df.select(mean("age")).show()
#df.select("Name").show(5)

Out[ 3 ]: 891

In [20]:
# df_test.filter(col('Fare').isNull()).show()
# df_test.filter(col('Pclass') ==3).show()
# df_test = df_test.withColumn("Fare",when((df_test["Fare"].isNull()), 8).otherwise(df_test["Fare"]))
# df = df.fillna({'Embarked':'S'})
# df = df.drop(col("Name"))
# df = df.drop("Cabin")

In [22]:
indexer = [StringIndexer(inputCol=column,outputCol=column+"_index").fit(titanic_df) for column in ["Sex","Embarked","Initial"]]

In [23]:
string_change_pipeline = Pipeline(stages=indexer)
preprocessing_pipeline = Pipeline()
mode_pipeline = Pipeline()


Pipeline([string_change_pipeline,preprocessing_pipeline,mode_pipeline])

In [24]:
%%time
titanic_df = pipeline.fit(titanic_df).transform(titanic_df)
%%time

In [25]:
titanic_df = titanic_df.drop("PassengerId","Name","Ticket","Cabin","Embarked","Sex","Initial")

In [26]:
titanic_df.columns[1:]

In [27]:
feature = VectorAssembler(inputCols=titanic_df.columns[1:],outputCol="features")
feature_vector = feature.transform(titanic_df)

In [28]:
feature_vector.select('features').show()

In [29]:
(training_data, test_data ) = feature_vector.randomSplit([0.8,0.2],seed=11)

In [30]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol='Survived',featuresCol='features') 
lrModel = lr.fit(training_data)

In [31]:
lr_prediction = lrModel.transform(test_data)

In [32]:
lr_prediction.select("features","rawPrediction","probability","prediction","Survived")
evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")

In [33]:
lr_accuracy = evaluator.evaluate(lr_prediction)
print("Accuracy of LogisticRegression is = %g"% (lr_accuracy))
print("Test Error of LogisticRegression = %g " % (1.0 - lr_accuracy))